In [21]:
import numpy as np
import plotly.graph_objects as go
import csv
from scipy import optimize
import sys,os
from sklearn.linear_model import LinearRegression

In [22]:
def read_data(file_path):
    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            column[0] = column[0].replace('(', '').replace("'" , '')
            column[2] = column[2].replace(')', '').replace("'" , '')
            column[1] = column[1].replace("'" , '')
            column[0] = float(column[0])
            column[1] = float(column[1])
            column[2] = float(column[2])
            data.append(column)
            # print(data)
        data = np.array(data)
    return data
        # print(data)

In [23]:
#Read energy in GeV of data files, save to list named energies
energies = []

#Read all files in data folder
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            energies.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies = np.array(energies, dtype=float)

In [24]:
results = []
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            # print(file)
            data = read_data(os.path.join(root, file))
            results.append(data)

In [25]:
#for i, array in enumerate(results):
    #print(f'data set{i} shape: {array.shape}')

In [26]:
def plot_pos(arrays, title, directory):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/pos/"
    os.makedirs(output_dir, exist_ok=True)

    for i, array in enumerate(arrays):
        # np.savetxt(f'{directory}/array_{i}.csv', array, delimiter=',')
        # print(f"plotting array {array[:,0]}")
        energy_str = str(energies[i]).replace('.', '_')
        fig = go.Figure(data=go.Scatter(x=array[:,0], y=array[:,1], mode='markers'))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        # fig.show()
        # fig.write_image('../media/plots/'+title+'.png')
        # Create the output directory if it doesn't already exist
        fig.write_image("../media/plots/1.1_pion-/pos/"+f"{energies[i]}"+ 'G'+".png")


        # print("../media/plots/"+f"{energy_str}"+ 'G'+".png")

In [27]:
plot_dir = '../media/plots'
plot_pos(results, 'GeV', plot_dir)

# Calculate emision angle

In [28]:
def momentum(E, m):
    return np.sqrt(E**2 - m**2)

In [29]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

In [30]:
radii = []
xc=[]
yc=[]
momenta = []
# print(type(energies))
for e in range(len(energies)):
    p = momentum(energies[e], 0.139)
    print(energies[e])
    momenta.append(p)
momenta_array = np.array(momenta)
for array in results:
    radii.append(fit_circle(array)[2])
    xc.append(fit_circle(array)[0])
    yc.append(fit_circle(array)[1])
radii = np.array(radii)
xc = np.array(xc)
yc = np.array(yc)
print("momenta array", momenta_array.shape, "energies", energies.shape, "radios ",radii.shape)
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
cherenkov_ang = np.array(cherenkov_angle)


processed_data = np.column_stack((energies, radii, momenta_array,cherenkov_ang))
print(processed_data.shape)

1.3
1.2
1.4
momenta array (3,) energies (3,) radios  (3,)
(3, 4)


In [31]:
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
    print(f'angle for {energies[i]} GeV: {theta}')
cherenkov_ang = np.array(cherenkov_angle)

print(cherenkov_ang)

angle for 1.3 GeV: 0.2454198540428831
angle for 1.2 GeV: 0.19274862761829764
angle for 1.4 GeV: 0.28358964689164456
[0.24541985 0.19274863 0.28358965]


In [32]:
def plot_circle(xc, yc, radii, arrays, energies):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/fit/"
    os.makedirs(output_dir, exist_ok=True)
    for i, array in enumerate(arrays):
        # Create the output directory if it doesn't already exist
        output_dir = "../media/plots/1.1_pion-/fit/"
        os.makedirs(output_dir, exist_ok=True)

        energy_str = str(energies[i]).replace('.', '_')
        theta_fit = np.linspace(0, 2*np.pi, 180)
        x_fit = xc[i] + (radii[i])*np.cos(theta_fit)
        y_fit = yc[i] + (radii[i])*np.sin(theta_fit)
        fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines', marker=dict(color='blue', opacity=0.8)))
        fig.add_trace(go.Scatter(x=array[:,0], y=array[:,1], mode='markers', marker=dict(color='violet', size=3, opacity=0.8)))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='y', yaxis_title='z')

        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="x",
                range=[-200, 200]
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="y",
                range=[-200, 200]
            )
        )
        #fig.show()
        fig.write_image("../media/plots/1.1_pion-/fit/"+f"{energies[i]}"+ 'G'+".png")



In [33]:
plot_circle(xc, yc, radii, results, energies)

In [34]:
def plot_momenta_cherenkov_angle(data):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/p/"
    os.makedirs(output_dir, exist_ok=True)
    fig = go.Figure(data=go.Scatter(x=data[:,2], y=data[:,3], mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
    fig.update_layout(title='Cherenkov angle vs momenta', xaxis_title='momenta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/1.1_pion-/p/"+"n_1_1"+".png")

In [35]:
plot_momenta_cherenkov_angle(processed_data)

In [36]:
def get_beta(energy, momentum):
    beta = []
    for i in range(len(energy)):
        beta.append(momentum[i]/energy[i])
    return np.array(beta)

In [37]:
beta = get_beta(energies, momenta)
cos_beta = np.cos(beta)
beta_1 = 1/beta
print(beta_1.shape, cos_beta.shape)

(3,) (3,)


In [38]:
def plot_cos_beta(cos,beta_1):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/beta/"
    fig = go.Figure(data=go.Scatter(x=cos, y=beta_1, mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
    fig.update_layout(title='Cherenkov angle vs beta', xaxis_title='beta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/1.1_pion-/beta/"+"n_1_1"+".png")

In [39]:
plot_cos_beta(cos_beta, beta_1)

In [40]:
#find the slope of plot
def get_slope(x, y):
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    reg = LinearRegression()
    fit = reg.fit(x, y)
    return reg

reg = get_slope(cos_beta, beta_1)
print("slope", reg.coef_[0][0])

slope 1.206987661568796
